In [1]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

In [2]:
!pip install import_ipynb==0.1.3
!pip install chess

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
import chess

import import_ipynb
import Game
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from RandomAlgorithm import make_random_move
from AlphaBetaAlgorithm import make_move_alphabeta, make_move_alphabeta_iterative_deepening




importing Jupyter notebook from Game.ipynb


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
importing Jupyter notebook from MinimaxAlgorithm.ipynb


importing Jupyter notebook from Util.ipynb


importing Jupyter notebook from Constants.ipynb


importing Jupyter notebook from RandomAlgorithm.ipynb


importing Jupyter notebook from AlphaBetaAlgorithm.ipynb


In [4]:
!ls /datasets/gaviota

g345-cp4.md5   knk.gtb.cp4    kqbkn.gtb.cp4  kqqkn.gtb.cp4  krkp.gtb.cp4
g345-cp4.sha1  knkn.gtb.cp4   kqbkp.gtb.cp4  kqqkp.gtb.cp4  krkr.gtb.cp4
kbbbk.gtb.cp4  knkp.gtb.cp4   kqbkq.gtb.cp4  kqqkq.gtb.cp4  krnkb.gtb.cp4
kbbkb.gtb.cp4  knnkb.gtb.cp4  kqbkr.gtb.cp4  kqqkr.gtb.cp4  krnk.gtb.cp4
kbbk.gtb.cp4   knnk.gtb.cp4   kqbnk.gtb.cp4  kqqnk.gtb.cp4  krnkn.gtb.cp4
kbbkn.gtb.cp4  knnkn.gtb.cp4  kqbpk.gtb.cp4  kqqpk.gtb.cp4  krnkp.gtb.cp4
kbbkp.gtb.cp4  knnkp.gtb.cp4  kqkb.gtb.cp4   kqqqk.gtb.cp4  krnkq.gtb.cp4
kbbkq.gtb.cp4  knnkq.gtb.cp4  kqk.gtb.cp4    kqqrk.gtb.cp4  krnkr.gtb.cp4
kbbkr.gtb.cp4  knnkr.gtb.cp4  kqkn.gtb.cp4   kqrbk.gtb.cp4  krnnk.gtb.cp4
kbbnk.gtb.cp4  knnnk.gtb.cp4  kqkp.gtb.cp4   kqrkb.gtb.cp4  krnpk.gtb.cp4
kbbpk.gtb.cp4  knnpk.gtb.cp4  kqkq.gtb.cp4   kqrk.gtb.cp4   krpkb.gtb.cp4
kbkb.gtb.cp4   knpkb.gtb.cp4  kqkr.gtb.cp4   kqrkn.gtb.cp4  krpk.gtb.cp4
kbk.gtb.cp4    knpk.gtb.cp4   kqnkb.gtb.cp4  kqrkp.gtb.cp4  krpkn.gtb.cp4
kbkn.gtb.cp4   knpkn.gtb.cp4  kqnk.gtb.cp4

In [5]:
# Simple manual test to make sure evaluate_move didn't break. Feel free to remove
#game = Game.Game(make_move_algorithm_black=make_move_alphabeta)
#game.play()

In [6]:
# Random
# Minimax
    # MinimaxMemo
# AlphaBeta
    # AlphaBeta_IterativeDeepening

#random_minimax = Game.Game(make_random_move, make_move_minimax)
#random_alpha = Game.Game(make_random_move, make_move_alphabeta)
#minimax_alpha = Game.Game(make_move_minimax, make_move_alphabeta)

#random_minimemo = Game.Game(make_random_move, make_move_minimax_memoization)
#random_alphadeep = Game.Game(make_random_move, make_move_alphabeta_iterative_deepening)
#minimemo_alpha = Game.Game(make_move_minimax_memoization, make_move_alphabeta)
#minimemo_alphadeep = Game.Game(make_move_minimax_memoization, make_move_alphabeta_iterative_deepening)
#alphadeep_mini = Game.Game(make_move_alphabeta_iterative_deepening, make_move_minimax)


#mini_minimemo = Game.Game(make_move_minimax, make_move_minimax_memoization)
#alpha_alphadeep = Game.Game(make_move_alphabeta, make_move_alphabeta_iterative_deepening)

In [11]:
game = Game.Game(make_move_alphabeta, make_move_alphabeta)
game.play()

DTM:


The game has ended. White is the winner!

Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)


In [8]:
random_alpha.play()

NameError: name 'random_alpha' is not defined

In [ ]:
minimax_alpha.play()

In [ ]:
random_minimemo.play()

In [ ]:
random_alphadeep.play()

In [ ]:
minimemo_alpha.play()

In [ ]:
minimemo_alphadeep.play()

In [ ]:
alphadeep_mini.play()

In [ ]:
mini_minimemo.play()

In [ ]:
alpha_alphadeep.play()

In [ ]:
#print(game.move_times)
#white_d = sum(game.move_times[chess.WHITE]) / len(game.move_times[chess.WHITE])
#black_d = sum(game.move_times[chess.BLACK]) / len(game.move_times[chess.BLACK])

#print(white_d)
#print(black_d)

In [ ]:
game = Game.Game(make_move_alphabeta)
game.play()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>